In [103]:
import jieba, os, json, codecs
import pandas as pd
from gensim import corpora, models, similarities, matutils
from sklearn.preprocessing  import LabelEncoder

In [31]:
# walk = os.walk('./baike_topic')
# for root, dirs, files in walk:
#     for name in files:
#         f = open(os.path.join(root, name), 'r')
#         raw = f.read()
#         word_list = list(jieba.cut(raw, cut_all = False))
#         train_set.append(word_list)

In [72]:
def load_data():
    train_list = []
    for line in open('../../data/train_clean.json', 'r'):
        train_list.append(json.loads(line))
    train = pd.DataFrame(train_list)
    
    #train_work = train[names[-1]]
    test_list = []
    for line in open('../../data/test_clean.json', 'r'):
        test_list.append(json.loads(line))
    test = pd.DataFrame(test_list)
    le = LabelEncoder()
    train['gender'] = le.fit_transform(train['gender'])
    names =  train.columns
    le = LabelEncoder()
    test['gender'] = le.fit_transform(test['gender'])
    del train['_id']
    del test['_id']
    train = train.fillna(0)
    test = test.fillna(0)
    #test['age'] = test['age'].apply(lambda x : int(x.replace(u'岁','').encode('ascii')))
    return train, test

def data_transform():
    train, test = load_data()
    train_set = []
    #print train['major'].apply(lambda x : type(x))
    train['major_cut'] = train['major'].apply(lambda x : ['none'] if isinstance( x, int ) else list(jieba.cut(x, cut_all = True)))
    return train['major_cut']


In [113]:
train,test = load_data()

In [117]:
name=  train.columns[-1]
size_set = train[name].apply(lambda x : [x[i]['size'] for i in range(len(x))])
sum([ 1 if x[0]== x[1] else 0 for x in size_set])/1.0/len(size_set)

0.36787142857142857

In [73]:
train_set = data_transform()

In [121]:
sum([ 1 if abs(x[0]-x[1]) <=3 else 0 for x in size_set])/1.0/len(size_set)

0.9071571428571429

In [74]:
train_set

0                                           [电子, 电子商务, 商务]
1                          [计算, 计算机, 计算机科学, 算机, 科学, 与, 技术]
2                  [计算, 计算机, 计算机信息, 算机, 机信息, 信息, 信息管理, 管理]
3                                                   [none]
4                            [计算, 计算机, 计算机软件, 算机, 机软件, 软件]
5                                          [信息, 与, 计算, 科学]
6                                          [信息, 与, 计算, 科学]
7                                           [软件, 软件工程, 工程]
8                                 [电子, 电子信息, 信息, 信息工程, 工程]
9                          [计算, 计算机, 计算机科学, 算机, 科学, 与, 技术]
10                         [计算, 计算机, 计算机科学, 算机, 科学, 与, 技术]
11                                                  [none]
12                                                  [none]
13                         [机械, 机械设计, 设计, 制造, 及其, 自动, 自动化]
14                                [电子, 电子信息, 信息, 信息工程, 工程]
15                                                [大气, 科学]
16                                          [通信, 通信工程, 工

In [75]:
dic = corpora.Dictionary(train_set)

In [76]:
corpus = [dic.doc2bow(text) for text in train_set]

In [77]:
tfidf = models.TfidfModel(corpus)

In [78]:
corpus_tfidf = tfidf[corpus]

In [105]:
lda = models.LdaModel(corpus_tfidf, id2word = dic, iterations=50, num_topics = 50)
corpus_lda = lda[corpus_tfidf]

In [106]:
for i in range(0, 50):
    print lda.print_topic(i)

0.105*理学 + 0.103*心理 + 0.100*心理学 + 0.098*成型 + 0.085*及 + 0.066*车辆 + 0.052*用心 + 0.047*工程 + 0.042*机械工程 + 0.040*运动
0.269*专业 + 0.172*文秘 + 0.150*酒店 + 0.071*社会 + 0.045*社会工作 + 0.045*工作 + 0.034*管理 + 0.026*计算机专业 + 0.009*西班牙 + 0.009*西班牙语
0.186*网络 + 0.176*网络工程 + 0.129*药学 + 0.075*工程 + 0.061*投资 + 0.047*中药 + 0.037*中药学 + 0.035*媒体 + 0.027*数字 + 0.026*学生
0.088*数学 + 0.083*公共事业 + 0.082*共事 + 0.082*事业 + 0.082*公共 + 0.042*医学 + 0.040*化工 + 0.033*朝鲜 + 0.033*朝鲜语 + 0.028*应用
0.300* + 0.039*税务 + 0.038*农林 + 0.031*方向 + 0.029*社会学 + 0.029*学前教育 + 0.029*学前 + 0.028*教育 + 0.023*社会 + 0.020*经济
0.352*会计 + 0.317*会计学 + 0.309*计学 + 0.004*高中 + 0.002*航运 + 0.001*外科 + 0.001*业务 + 0.001*业务管理 + 0.001*国际航运 + 0.001*航运业
0.465*企业 + 0.154*工商企业 + 0.149*管理 + 0.077*工商 + 0.021*国际金融 + 0.006*会 + 0.004*供电 + 0.003*企业财务 + 0.003*金融法 + 0.003*现
0.378*旅游 + 0.131*管理 + 0.075*无 + 0.069*汽车 + 0.053*维修 + 0.030*共管 + 0.024*车检 + 0.023*会展 + 0.022*公共 + 0.022*与
0.381*自动 + 0.379*自动化 + 0.029*电气 + 0.022*与 + 0.014*安装 + 0.013*电子 + 0.013*Business + 0.011*工程 + 0.006*安装工 + 0.00

In [112]:
numpy_matrix = matutils.corpus2dense(corpus, num_terms=10000)
#matutils.corpus2dense(corpus_lda, corpus_tfidf.num_terms, corpus_tfidf.num_docs)

KeyboardInterrupt: 